##### Import

In [14]:
import warnings
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm
import shap
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.pipeline import Pipeline
import os
import gc
import sys

# Filter out warning messages
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

# Set seaborn style
sns.set_style('whitegrid')

# Add the parent directory to sys.path
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Index and deciles for data slicing
idx = pd.IndexSlice



from pathlib import Path

# Paths to the downloaded datasets, model, and hyperparameters
data_dir = Path('data/')
model_dir = Path('models/')
best_hyperparams_dir = Path('best_hyperparams/')
study_dir = Path('study/')

# Create directories if they do not exist
data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)
best_hyperparams_dir.mkdir(parents=True, exist_ok=True)
study_dir.mkdir(parents=True, exist_ok=True)

In [15]:
# from pathlib import Path
# import pandas as pd
# from utils import rank_stocks_and_quantile
# # UNSEEN_KEY = '/data/YEAR_20220803_20230803'
# top = 250  # parameters -> papermill
# DATA_STORE = Path(f'data/{top}_dataset.h5')
# with pd.HDFStore(DATA_STORE) as store:
#     # unseen = store[UNSEEN_KEY]
#     print(store.keys())

In [16]:
"""
Process Large Financial Datasets from HDF5 Format.

This script reads, processes, and normalizes financial datasets stored in an HDF5 format.
The primary processing steps involve converting data types, handling infinite values, and
scaling the dataset. The MinMaxScaler, computed from the entire dataset, is employed for normalization.
Once data processing is complete, stocks are ranked, and quantiles are determined in post-processing.

Attributes:
    - top (int): Number of top stocks to consider.
    - DATA_STORE (Path): Path to the HDF5 file containing the datasets.
    - dataset_keys (list of str): Keys identifying which datasets to process in the HDF5 store.
    - target_string (str): Target column identifier for post-processing.
    - CHUNK_SIZE (int): Size of chunks in which data is read and processed.

Functions:
    - convert_dtype(chunk, feature_columns, dtype='float32'): Converts dtype of specified columns in a chunk.
    - handle_infinite_values(chunk, feature_columns): Handles infinite values in a chunk.
    - process_chunk(chunk, feature_columns, scaler=None): Process a single chunk with optional normalization.

Workflow:
    1. Set parameters and paths.
    2. Define utility functions.
    3. Identify features and target columns from the first chunk.
    4. Determine the MinMaxScaler using all chunks in the dataset.
    5. Process and concatenate chunks to form the dataset.
    6. Rank stocks and compute quantiles in post-processing.
"""

import gc
import numpy as np
import pandas as pd
from pathlib import Path
from utils import rank_stocks_and_quantile
from sklearn.preprocessing import MinMaxScaler

# Parameters and data paths
TOP = top = 500
DATA_STORE = Path(f'data/{top}_dataset.h5')
dataset_keys = [
    '/data/YEAR_20200930_20220802',
    '/data/YEAR_20181024_20200929',
    '/data/YEAR_20161116_20181023',
    '/data/YEAR_20141210_20161115'
]
target_string = 'TARGET_ret_fwd'
CHUNK_SIZE = 50000

def convert_dtype(chunk, feature_columns, dtype='float32'):
    """Converts the datatype of the specified columns."""
    chunk[feature_columns] = chunk[feature_columns].astype(dtype)
    return chunk

def handle_infinite_values(chunk, feature_columns):
    """Handle infinite values by replacing them with the maximum finite value."""
    max_val = np.finfo('float32').max
    chunk[feature_columns] = chunk[feature_columns].replace([np.inf, -np.inf], max_val)
    return chunk

def process_chunk(chunk, feature_columns, scaler=None):
    """Process a single chunk of data."""
    chunk = convert_dtype(chunk, feature_columns)
    chunk = handle_infinite_values(chunk, feature_columns)
    
    # Normalize with scaler if provided
    if scaler:
        chunk[feature_columns] = scaler.transform(chunk[feature_columns])
    
    return chunk

# Identify features and targets based on the first chunk
with pd.HDFStore(DATA_STORE) as store:
    first_chunk = store.select(dataset_keys[0], stop=CHUNK_SIZE)
    features = [col for col in first_chunk.columns if col.startswith('FEATURE_')]
    target = [col for col in first_chunk.columns if col.startswith('TARGET_')]

# Determine the scaler using the entire dataset for the identified features
scaler = MinMaxScaler()

for key in dataset_keys:
    with pd.HDFStore(DATA_STORE) as store:
        for chunk in store.select(key, chunksize=CHUNK_SIZE):
            # Convert dtype and handle infinite values
            chunk = convert_dtype(chunk, features)
            chunk = handle_infinite_values(chunk, features)
            scaler.partial_fit(chunk[features])

# Process and concatenate chunks
dataset = pd.DataFrame()
for key in dataset_keys:
    with pd.HDFStore(DATA_STORE) as store:
        for chunk in store.select(key, chunksize=CHUNK_SIZE):
            processed_chunk = process_chunk(chunk, features, scaler)
            dataset = pd.concat([dataset, processed_chunk], ignore_index=False)
            del processed_chunk
            gc.collect()

# Post-processing steps
dataset = rank_stocks_and_quantile(dataset, target_substring=target_string)
dataset.index.set_levels(dataset.index.levels[0].tz_localize(None), \
    level=0, inplace=True)

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from joblib import Parallel, delayed

PADDING_VALUE = -1
MAX_LEN = None  # If you have a predefined value, set it here; otherwise, it gets calculated automatically.

def pad_sequence(inputs, padding_value=-1, max_len=None):
    if max_len is None:
        max_len = max([input.shape[0] for input in inputs])
    padded_inputs = []
    masks = []
    for input in inputs:
        pad_len = max_len - input.shape[0]
        padded_input = F.pad(input, (0, 0, 0, pad_len), value=padding_value)
        mask = torch.ones((input.shape[0], 1), dtype=torch.float)
        masks.append(
            torch.cat((mask, torch.zeros((pad_len, 1), dtype=torch.float)), dim=0)
        )
        padded_inputs.append(padded_input)
    return torch.stack(padded_inputs), torch.stack(masks)

def convert_to_torch(timestamp, data):
    feature_names = [col for col in data.columns if col.startswith('FEATURE_')]
    target_names = [col for col in data.columns if col.startswith('TARGET_')]
    
    inputs = torch.from_numpy(
                data[feature_names].values.astype(np.float32))
    labels = torch.from_numpy(
                data[target_names].values.astype(np.float32))

    padded_inputs, masks_inputs = pad_sequence(
            [inputs], padding_value=PADDING_VALUE, max_len=MAX_LEN)
    padded_labels, masks_labels = pad_sequence(
            [labels], padding_value=PADDING_VALUE, max_len=MAX_LEN)

    return {
        timestamp: (
            padded_inputs,
            padded_labels,
            masks_inputs,
            target_names
        )
    }

def get_era2data(df):
    # Group by the Timestamp index (level=0)
    res = Parallel(n_jobs=-1, prefer="threads")(
        delayed(convert_to_torch)(timestamp, data)
        for timestamp, data in tqdm(df.groupby(level=0)))
    
    era2data = {}
    for r in tqdm(res):
        era2data.update(r)
    return era2data

# Assuming DataFrame is named "dataset": testing the function
timestamp2data_dataset = get_era2data(dataset)

  0%|          | 0/1924 [00:00<?, ?it/s]

100%|██████████| 1924/1924 [00:00<00:00, 2049223.18it/s]


In [18]:
import torch
import torch.nn as nn

def pearsonr(x, y):
    xm, ym = x - x.mean(), y - y.mean()
    r_num = torch.sum(xm * ym)
    r_den = torch.sqrt(torch.sum(xm ** 2) + 1e-10) * torch.sqrt(torch.sum(ym ** 2) + 1e-10)
    correlation = r_num / r_den
    return correlation.requires_grad_()  # Ensure that the returned tensor requires gradients

def spearmanr(x, y):
    rank_x = x.argsort().argsort().float()
    rank_y = y.argsort().argsort().float()
    return pearsonr(rank_x, rank_y)

def pairwise_ranking_loss(outputs, target_labels, masks_inputs):
    sorted_indices = torch.argsort(target_labels, dim=-1, descending=True)
    sorted_outputs = torch.gather(outputs, -1, sorted_indices)
    
    diff_vector = sorted_outputs[:, 1:] - sorted_outputs[:, :-1]
    sigmoid_diff = 1.0 / (1.0 + torch.exp(-diff_vector))
    loss = -torch.log(torch.clamp(sigmoid_diff, min=1e-10, max=1-1e-10))
    
    min_dim = min(masks_inputs.shape[1], loss.shape[1])
    masked_loss = loss[:, :min_dim] * masks_inputs[:, :min_dim]
    
    return torch.sum(masked_loss)

def calculate_loss(outputs, criterion, target_labels, masks_inputs, alpha_mse=0.5, alpha_corr=1.0, alpha_rank=1.0):
    
    # 1. Print basic info
    # print(f"Outputs Range: {outputs.min().item()}, {outputs.max().item()}")
    # print(f"Target Labels Range: {target_labels.min().item()}, {target_labels.max().item()}")
    # print(f"Masks Range: {masks_inputs.min().item()}, {masks_inputs.max().item()}")
    
    # Calculating the MSE loss
    mse_main = criterion(outputs * masks_inputs, target_labels * masks_inputs)
    # print(f"MSE Loss: {mse_main.item()}")
    
    # Calculating the Spearman Correlation
    non_zero_mask = masks_inputs.view(-1).nonzero().squeeze()
    spearman_corr = spearmanr(outputs[0][:, 0][non_zero_mask], target_labels[0][:, 0][non_zero_mask])
    # print(f"Spearman Correlation: {spearman_corr.item()}")

    # Calculating the Ranking Loss
    ranking_loss = pairwise_ranking_loss(outputs, target_labels, masks_inputs)
    # print(f"Ranking Loss: {ranking_loss.item()}")

    # Using alpha values to weight the losses and normalize them
    losses = [mse_main, -spearman_corr, ranking_loss]
    alphas = [alpha_mse, alpha_corr, alpha_rank]
    weights = [alpha / (loss + 1e-10) for alpha, loss in zip(alphas, losses)]
    normalized_weights = [weight / sum(weights) for weight in weights]
    
    combined_loss = sum(w * l for w, l in zip(normalized_weights, losses))
    
    return combined_loss.requires_grad_(), mse_main, spearman_corr


In [19]:
# Training loop
def train_on_batch(model, criterion, optimizer, batch, lookahead):
    inputs, labels, masks_inputs, target_names = batch

    # Get index for specific label dynamically
    specific_label_name = f'TARGET_ret_fwd_{lookahead:02d}d_rank_quantiled'
    specific_label_index = target_names.index(specific_label_name)

    # Use that index to fetch the specific column
    labels = labels[:, :, specific_label_index].unsqueeze(2)

    # print('labels shape: ', labels.shape)

    # Zero the parameter gradients
    optimizer.zero_grad()

    outputs = model(inputs / 4.0, masks_inputs)

    # print("Output shape: ", outputs.shape)

    assert labels.shape == outputs.shape, \
        f"Shape mismatch: labels {labels.shape}, outputs {outputs.shape}"

    loss, _mse, _corr = calculate_loss(outputs, criterion, labels, masks_inputs)
    
    loss.backward()
    optimizer.step()
    
    return loss.item(), _mse.item(), _corr.item()

def evaluate_on_batch(model, criterion, batch, lookahead):
    inputs, labels, masks_inputs, target_names = batch

    # Get index for specific label dynamically
    specific_label_name = f'TARGET_ret_fwd_{lookahead:02d}d_rank_quantiled'
    specific_label_index = target_names.index(specific_label_name)

    # Use that index to fetch the specific column
    labels = labels[:, :, specific_label_index].unsqueeze(2)

    model.eval()
    with torch.no_grad():
        outputs = model(inputs / 4, masks_inputs)

        # print('output form eval: ', outputs.shape)

        assert labels.shape == outputs.shape, \
            f"Shape mismatch: labels {labels.shape}, outputs {outputs.shape}"

        loss, mse, corr = calculate_loss(outputs, criterion, labels, masks_inputs)

        # Assuming masks_inputs is of shape (1, 253, 1)
        non_zero_indices = masks_inputs.squeeze().nonzero().squeeze()

        # Gather values from outputs tensor using the non-zero indices
        preds = torch.gather(outputs.squeeze(), 0, non_zero_indices).cpu().numpy()

    return loss.item(), mse.item(), corr.item(), preds


def compute_fold_metrics(era_scores, weights=None):
    era_scores = pd.Series(era_scores)
    
    # Calculate metrics
    mean_correlation = np.mean(era_scores)
    std_deviation = np.std(era_scores)
    sharpe_ratio = mean_correlation / std_deviation
    max_dd = (era_scores.cummax() - era_scores).max()

    # Smart Sharpe
    smart_sharpe = mean_correlation \
        / (std_deviation + np.std(era_scores.diff()))
    
    # Autocorrelation
    autocorrelation = era_scores.autocorr()

    metrics = pd.Series({
        'mean_correlation': mean_correlation,
        'std_deviation': std_deviation,
        'sharpe_ratio': sharpe_ratio,
        'smart_sharpe': smart_sharpe,
        'autocorrelation': autocorrelation,
        'max_dd': max_dd,
        'min_correlation': era_scores.min(),
        'max_correlation': era_scores.max(),
    })

    if weights:
        normalized_metrics = (metrics - metrics.min()) / (metrics.max() - metrics.min())
        weighted_values = normalized_metrics.multiply(pd.Series(weights))
        metrics["weighted_score"] = weighted_values.sum()

    _ = gc.collect()

    return metrics

In [20]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, scheduler, \
                num_epochs, patience, train_loader, lookahead, \
                device, val_loader=None, is_lr_scheduler=True):
    best_score = float('-inf')  # Initialize with negative infinity since we want to maximize Sharpe ratio
    best_corr = None
    best_model_wts = None  # Changing from 'best_model' to avoid confusion with the model object
    all_val_scores = []
    all_val_outputs = {}
    no_improve_epoch = 0

    model = model.to(device)

    epoch_progress = tqdm(range(num_epochs), desc="Epochs", leave=False)

    for epoch in epoch_progress:
        total_loss = []
        total_corr = []

        # Training
        for era_num in tqdm(train_loader, desc="Training", leave=False):
            data = train_loader[era_num]
            batch = (data[0].to(device), data[1].to(device), data[2].to(device), data[3])
            
            loss, _mse, _corr = train_on_batch(model, criterion, optimizer, batch, lookahead)
            # print(loss)
            total_loss.append(loss)
            total_corr.append(_corr)

        # Adjust learning rate if is_lr_scheduler is True
        if is_lr_scheduler:
            scheduler.step()

        # Validation - Only if val_loader is provided
        if val_loader:
            val_total_loss = []
            val_total_corr = []
            val_total_outputs = {}

            with torch.no_grad():
                for era_num in tqdm(val_loader, desc="Validation", leave=False):
                    data = val_loader[era_num]
                    batch = (data[0].to(device), data[1].to(device), data[2].to(device), data[3])
                    
                    loss, _mse, _corr, outputs = evaluate_on_batch(model, criterion, batch, lookahead)
                    val_total_loss.append(loss)
                    val_total_corr.append(_corr)
                    val_total_outputs[era_num] = outputs

            all_val_scores.append(val_total_corr) 
            all_val_outputs.update(val_total_outputs)

            # Early stopping check based on Sharpe score
            current_score = np.mean(val_total_corr) / np.std(val_total_corr)  # Assuming Sharpe ratio here
            if current_score > best_score:
                best_score = current_score
                best_corr = val_total_corr.copy()
                best_model_wts = model.state_dict().copy()
                no_improve_epoch = 0
            else:
                no_improve_epoch += 1
                if no_improve_epoch >= patience:
                    epoch_progress.set_description(f'Early stopping at epoch {epoch+1}')
                    epoch_progress.refresh()
                    break

        torch.cuda.empty_cache()
        _ = gc.collect()

    if val_loader:  # If validation data was provided
        return best_model_wts, best_corr, all_val_scores
    else:  # If only training data was used without validation
        return model.state_dict(), None, None

In [21]:
import optuna
import mlflow
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from utils import CustomBackwardMultipleTimeSeriesCV
from model import Transformer
from model import RankPredictorNN
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


# Constants and hyperparameters
NUM_EPOCHS = 15
PATIENCE = 5
FEATURE_DIM = len(features)  # Assuming 'features' is defined elsewhere in your code
OUTPUT_DIM = 1
NUM_TRAIL = 25
device = "cuda" if torch.cuda.is_available() else "cpu"

# # Choose model
# model = Transformer(
#     input_dim=FEATURE_DIM,
#     d_model=hidden_dim,
#     output_dim=OUTPUT_DIM,
#     num_heads=num_heads,
#     num_layers=num_layers,
# ).to(device)

weights = {
    'mean_correlation': 0.0,
    'std_deviation': -0.025, # Mild penalty for higher volatility
    'sharpe_ratio': 0.95,    # Primary objective, so highest weight
    'smart_sharpe': 0.075,   # Supplementary to Sharpe Ratio but considering autocorrelation
    'autocorrelation': -0.1, # Penalize strategies showing signs of overfitting
    'max_dd': -0.1,          # Major risk metric, negative to penalize higher drawdowns
    'min_correlation': 0.0,
    'max_correlation': 0.0,
}

def objective(trial, dataset, device):  # Placeholder for dataset
    print(f"\n--- Starting Trial: {trial.number + 1} ---")

    # Suggest hyperparameters
    train_length_multiplier = trial.suggest_int('train_length_multiplier', 10, 15)
    val_period_length = trial.suggest_categorical('val_period_length', [21, 42, 63])
    lookahead = trial.suggest_categorical('lookahead', [1, 5, 21])
    num_heads = trial.suggest_int("num_heads", 1, 5)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256, step=2)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)

    print(f"Hyperparameters for this trial: {trial.params}")


    # Initialize CV and other variables
    cv = CustomBackwardMultipleTimeSeriesCV(dataset,
                                    train_period_length=int(21 * train_length_multiplier),
                                    test_period_length=val_period_length,
                                    lookahead=lookahead, date_idx='date')

    cv.update_lookahead(lookahead)
    fold_weighted_scores = []
    for train_idx, test_idx in cv:
        # Choose model
        model = Transformer(
            input_dim=FEATURE_DIM,
            d_model=hidden_dim,
            output_dim=OUTPUT_DIM,
            num_heads=num_heads,
            num_layers=num_layers).to(device)

        # # Initialize model, loss, optimizer
        # model = RankPredictorNN(input_dim=FEATURE_DIM, output_dim=OUTPUT_DIM).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = StepLR(optimizer, step_size=100, gamma=0.1)

        # Prepare data batches
        train_data = dataset.iloc[train_idx]
        test_data = dataset.iloc[test_idx]
        train_batches = get_era2data(train_data)
        validation_batches = get_era2data(test_data)

        # Train and validate model
        _, val_corr_on_fold, _ = train_model(
            model, criterion, optimizer, scheduler, NUM_EPOCHS, PATIENCE,
            train_batches, lookahead, device, validation_batches, is_lr_scheduler=True
        )

        # print(val_corr_on_fold)

        # Compute metrics
        scores_on_fold = compute_fold_metrics(val_corr_on_fold)

        # Normalize and weight scores
        normalized_scores = (scores_on_fold - scores_on_fold.min()) \
            / (scores_on_fold.max() - scores_on_fold.min())
        weighted_scores_on_fold = normalized_scores.multiply(pd.Series(weights))

        # Append to list
        fold_weighted_scores.append(weighted_scores_on_fold.sum())

    # Calculate overall score
    overall_score = np.mean(fold_weighted_scores)
    # print('==================')
    # print('Overall score: ', overall_score)
    # print('==================')

    # Log metrics
    with mlflow.start_run():
        mlflow.log_params(trial.params)
        mlflow.log_metric("avg_score_across_folds", overall_score)

    return -overall_score if not np.isnan(overall_score) else 1e-9

def callback(study, trial):
    print(f"\n--- Trial {trial.number + 1} finished ---")
    print(f"Value: {trial.value} and parameters: {trial.params}")
    
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    
    if completed_trials:
        best_trial_number = study.best_trial.number + 1  # Adding 1 to align with your display
        print(f"Best is trial {best_trial_number} with value: {study.best_trial.value}\n")
    else:
        print("No successful trials yet.\n")

study_dir = "/home/sayem/Desktop/Project/study"
# study = optuna.create_study(study_name='Maximizing the Sharpe', direction='minimize',
#                             storage=f'sqlite:///{study_dir}/study.db', load_if_exists=True)
study = optuna.create_study(study_name='Maximizing the Sharpe', \
    direction='minimize', load_if_exists=True)
# study.optimize(objective, n_trials=NUM_TRAIL, callbacks=[callback])
study.optimize(lambda trial: objective(trial, dataset, device), n_trials=NUM_TRAIL, callbacks=[callback])

[I 2023-10-11 00:11:58,104] A new study created in memory with name: Maximizing the Sharpe



--- Starting Trial: 1 ---
Hyperparameters for this trial: {'train_length_multiplier': 10, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 4, 'hidden_dim': 158, 'num_layers': 4, 'learning_rate': 0.0003473467864674847}


Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:00<00:00, 607450.92it/s]
[I 2023-10-11 00:22:58,943] Trial 0 finished with value: -0.9848623647133862 and parameters: {'train_length_multiplier': 10, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 4, 'hidden_dim': 158, 'num_layers': 4, 'learning_rate': 0.0003473467864674847}. Best is trial 0 with value: -0.9848623647133862.



--- Trial 1 finished ---
Value: -0.9848623647133862 and parameters: {'train_length_multiplier': 10, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 4, 'hidden_dim': 158, 'num_layers': 4, 'learning_rate': 0.0003473467864674847}
Best is trial 1 with value: -0.9848623647133862


--- Starting Trial: 2 ---
Hyperparameters for this trial: {'train_length_multiplier': 13, 'val_period_length': 63, 'lookahead': 1, 'num_heads': 5, 'hidden_dim': 218, 'num_layers': 3, 'learning_rate': 0.0017896948297652798}


...


--- Trial 23 finished ---
Value: -0.9852045439296993 and parameters: {'train_length_multiplier': 13, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 4, 'hidden_dim': 66, 'num_layers': 2, 'learning_rate': 0.0010230449330602869}
Best is trial 7 with value: -0.9871639090080142


--- Starting Trial: 24 ---
Hyperparameters for this trial: {'train_length_multiplier': 13, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 5, 'hidden_dim': 66, 'num_layers': 2, 'learning_rate': 0.0014531514529634428}


100%|██████████| 21/21 [00:00<00:00, 620284.39it/s]
[I 2023-10-11 02:22:50,433] Trial 23 finished with value: -0.9808941600899868 and parameters: {'train_length_multiplier': 13, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 5, 'hidden_dim': 66, 'num_layers': 2, 'learning_rate': 0.0014531514529634428}. Best is trial 6 with value: -0.9871639090080142.



--- Trial 24 finished ---
Value: -0.9808941600899868 and parameters: {'train_length_multiplier': 13, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 5, 'hidden_dim': 66, 'num_layers': 2, 'learning_rate': 0.0014531514529634428}
Best is trial 7 with value: -0.9871639090080142


--- Starting Trial: 25 ---
Hyperparameters for this trial: {'train_length_multiplier': 14, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 3, 'hidden_dim': 100, 'num_layers': 2, 'learning_rate': 0.0024457388375685573}


100%|██████████| 21/21 [00:00<00:00, 537075.51it/s]
[I 2023-10-11 02:27:00,712] Trial 24 finished with value: -0.9783406131475749 and parameters: {'train_length_multiplier': 14, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 3, 'hidden_dim': 100, 'num_layers': 2, 'learning_rate': 0.0024457388375685573}. Best is trial 6 with value: -0.9871639090080142.



--- Trial 25 finished ---
Value: -0.9783406131475749 and parameters: {'train_length_multiplier': 14, 'val_period_length': 21, 'lookahead': 21, 'num_heads': 3, 'hidden_dim': 100, 'num_layers': 2, 'learning_rate': 0.0024457388375685573}
Best is trial 7 with value: -0.9871639090080142



In [22]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
# After all trials have finished, retrieve the best trial's parameters
best_params = study.best_trial.params

# # Create the best model using the Transformer
# best_model = Transformer(
#     input_dim=FEATURE_DIM,
#     d_model=best_params["hidden_dim"],
#     output_dim=OUTPUT_DIM,
#     num_heads=best_params["num_heads"],
#     num_layers=best_params["num_layers"]
# ).to(device)

# Below is the SimpleNN code, commented out:
best_model = SimpleNN(input_dim=FEATURE_DIM, output_dim=OUTPUT_DIM).to(device)

# Train the best model on the entire dataset
criterion = nn.MSELoss()
lr = best_params['learning_rate']
optimizer = optim.Adam(best_model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=100, gamma=0.1)

# Assuming get_era2data() can handle the entire dataset
all_batches = get_era2data(dataset)  

# You might need to adjust/train_model to handle no validation set or adjust accordingly.
_, _, _ = train_model(
    best_model, criterion, optimizer, scheduler, NUM_EPOCHS, PATIENCE, 
    all_batches, None, is_lr_scheduler=True  # Assuming train_model can handle None for validation_batches
)

# Saving the model
model_name = best_model.__class__.__name__
lookahead = best_params.get("lookahead", "NA")
filename = f"{top}_{model_name}_{target_string}_{lookahead:02d}d_rank_quantiled.pkl"
file_path = os.path.join(model_dir, filename)

save_data = {
    'model_type': 'Transformer',
    'model_state_dict': best_model.state_dict(),
    'trial_params': best_params
}
torch.save(save_data, file_path)

In [ ]:
# Loading the saved data
loaded_data = torch.load(file_path)

# Create the correct model based on the saved type
if loaded_data['model_type'] == 'Transformer':
    model = Transformer(
        input_dim=FEATURE_DIM,
        d_model=loaded_data['trial_params']["hidden_dim"],
        output_dim=OUTPUT_DIM,
        num_heads=loaded_data['trial_params']["num_heads"],
        num_layers=loaded_data['trial_params']["num_layers"]
    ).to(device)
else:
    model = SimpleNN(input_dim=FEATURE_DIM, output_dim=OUTPUT_DIM).to(device)

# Load the saved parameters into the model
model.load_state_dict(loaded_data['model_state_dict'])

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.optim.lr_scheduler import StepLR
# from model import RankPredictorNN  # Assuming this is where your SimpleNN class is defined
# import os
# from model import Transformer
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# # Constants and hyperparameters
# NUM_EPOCHS = 150
# PATIENCE = 5
# FEATURE_DIM = len(features)  # Assuming 'features' is defined elsewhere in your code
# OUTPUT_DIM = 1
# NUM_TRAIL = 25
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Best parameters from Optuna study
# best_params = {
#     'train_length_multiplier': 14,
#     'val_period_length': 63,
#     'lookahead': 5,
#     'num_heads': 5,  # Updated to match with the Transformer definition below
#     'hidden_dim': 208,
#     'num_layers': 10,
#     'learning_rate': 0.002
# }

# # Choose model
# best_model = Transformer(
#     input_dim=FEATURE_DIM,
#     d_model=best_params['hidden_dim'],
#     output_dim=OUTPUT_DIM,
#     num_heads=best_params['num_heads'],
#     num_layers=best_params['num_layers'],
# ).to(device)


# # # Initialize the best model using SimpleNN
# # best_model = RankPredictorNN(input_dim=FEATURE_DIM, \
# #     output_dim=OUTPUT_DIM).to(device)

# # Initialize loss function, optimizer, and learning rate scheduler
# criterion = nn.MSELoss()
# optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])
# scheduler = StepLR(optimizer, step_size=100, gamma=0.1)


# # label = f'TARGET_ret_fwd_{params["lookahead"]:02d}d_rank_quantiled'
# # Assuming get_era2data() can handle the entire dataset and returns a DataLoader
# all_batches = get_era2data(dataset)  # Replace this with your actual data loading function

# # Training Loop
# # Training Loop
# for epoch in range(NUM_EPOCHS):
#     best_model.train()
    
#     total_loss = 0.0
#     total_mse = 0.0
#     total_corr = 0.0
    
#     # Define the specific label using lookahead
#     for timestamp, (inputs, labels, masks_inputs, target_names) in all_batches.items():
        
#         # Move tensors to the desired device
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         masks_inputs = masks_inputs.to(device)

#         # Get index for specific label dynamically
#         specific_label_name = f'TARGET_ret_fwd_{best_params["lookahead"]:02d}d_rank_quantiled'
#         specific_label_index = target_names.index(specific_label_name)

#         # Use that index to fetch the specific column
#         labels = labels[:, :, specific_label_index].unsqueeze(2)
#         # print(f"Target Labels Range from training loop: {labels.min().item()}, {labels.max().item()}")

#         # Zero the parameter gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = best_model(inputs / 4, masks_inputs)
#         # print(outputs)
        
#         # Asserting that shapes of labels and outputs match
#         assert labels.shape == outputs.shape, \
#             f"Shape mismatch: labels {labels.shape}, outputs {outputs.shape}"

#         # Compute loss using the custom loss function
#         loss, mse, corr = calculate_loss(outputs, criterion, \
#                     labels, masks_inputs)

#         # print(f"Current batch loss: {loss.item()}, Current batch MSE: {mse.item()}, Current batch Correlation: {corr.item()}")
#         total_loss += loss.item()
#         total_mse += mse.item()
#         total_corr += corr.item()

#         # print(f"Accumulated Total Loss after this batch: {total_loss}, Accumulated Total MSE after this batch: {total_mse}, Accumulated Total Correlation after this batch: {total_corr}")

            
#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()
#         # break
        
#     # Step the learning rate scheduler
#     scheduler.step()

#     # # At the end of the training loop:
#     # print(f"Total loss: {total_loss}, Total MSE: {total_mse}, Total Correlation: {total_corr}, Number of batches: {len(all_batches)}")

#     avg_loss = total_loss / len(all_batches)
#     avg_mse = total_mse / len(all_batches)
#     avg_corr = total_corr / len(all_batches)
        
#     print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}, MSE: {avg_mse:.4f}, Correlation: {avg_corr:.4f}")

#     # break


# print("Training complete.")